In [ ]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from pysindy import SINDy
plot_kws = dict(alpha=0.7, linewidth=0.3)
pal = sns.color_palette("Set1")
from pysindy.differentiation import FiniteDifference
from pysindy.feature_library import PolynomialLibrary
from pysindy.optimizers import SR3,SSR,FROLS,TrappingSR3,STLSQ

import skccm as ccm
from skccm.utilities import train_test_split
import csv

In [ ]:
def lorenz(p,t,s,r,b):

    x,y,z=p
    return [s*(y-x),x*(r-z)-y,x*y-b*z]
def rossler(p,t,f1,f2,w12,w21):
    x1,y1,z1,x2,y2,z2=p
 
    return [-f1*y1-z1+w12*z2,f1*x1+0.1*y1,0.1+z1*(x1-14),-f2*y2-z2+w21*z1,f2*x2+0.1*y2,0.1+z2*(x2-14)] #返回dx/dt,dy/dt,dz/dt


t = np.arange(0,100,0.05)


track1 = integrate.odeint(lorenz,[-8,5,10],t,args=(10,28.0,3))

track2= integrate.odeint(lorenz,[-7,4,9],t,args=(12,30.0,4))
track3 = integrate.odeint(rossler,[-3,-10,5,3,10,-5],t,args=(1.99,1.85,1.2,0.3))
X=np.hstack((track1,track2,track3[:,[1,4]]))

In [ ]:
import seaborn as sns
import csv
print(X.shape)
score=np.zeros((8,8))
lag=5
embed=3
for i in range(8):
    for j in range(i,8):
        if i==j:
            continue

        e1 = ccm.Embed(X[:,i])
        e2 = ccm.Embed(X[:,j])

        X1 = e1.embed_vectors_1d(lag,embed)
        X2 = e2.embed_vectors_1d(lag,embed)
        x1tr, x1te, x2tr, x2te = train_test_split(X1,X2, percent=.75)
        CCM_ = ccm.CCM() #initiate the class
#library lengths to test
        len_tr = len(x1tr)
        lib_lens = np.arange(10, len_tr, len_tr/20, dtype='int')
#test causation
        CCM_.fit(x1tr,x2tr)
        x1p, x2p = CCM_.predict(x1te, x2te,lib_lengths=lib_lens)
        sc1,sc2 = CCM_.score()
        score[i,j]=np.max(np.abs(sc1))
        score[j,i]=np.max(np.abs(sc2))
print(score)

heat=sns.heatmap(data=score,cmap='coolwarm')
heat_fig = heat.get_figure()


In [ ]:


feature_library = PolynomialLibrary(degree=2)
feature_library.fit(X)
feature_library.get_feature_names()
optimizer = STLSQ(threshold=0.025)
model = SINDy(
    differentiation_method= FiniteDifference(),
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=['x1','y1','z1',"x2", "y2","z2",'ry1','ry2'],

)

model.fit(X, t=0.05)
model.print()
score1 = model.score(X,t=0.05)
print(score1)

In [ ]:
from scipy import interpolate
list_old=np.linspace(0,100,2000)
list_new=np.linspace(0,100,10000)
print(X.shape)
f_linear_1=interpolate.interp1d(list_old,X[:,3])
f_linear_2=interpolate.interp1d(list_old,X[:,4])
f_linear_3=interpolate.interp1d(list_old,X[:,5])
result_x_2=f_linear_1(list_new)
result_y_2=f_linear_2(list_new)
result_z_2=f_linear_3(list_new)
sys_1 = np.stack((result_x_2, result_y_2,result_z_2), axis=1)

feature_library = PolynomialLibrary(degree=2)
feature_library.fit(X)
feature_library.get_feature_names()
optimizer = STLSQ(threshold=0.025)
model = SINDy(
    differentiation_method= FiniteDifference(),
    feature_library=feature_library,
    optimizer=optimizer,
    feature_names=["x2", "y2","z2","ry1"],
    choose_prune=False
)
print(sys_1.shape)
model.fit(sys_1, t=0.01)
model.print()
score5 = model.score(sys_1,t=0.01)
print(score5)
